In [ ]:
using CSV, DataFrames
using FileIO, Dates
using Plots
using StatsBase

In [ ]:
include("./src/cubes.jl")
include("./src/network.jl")
include("./src/motifs_discovery.jl")
include("./src/motifs_analysis.jl")

### Read data

In [ ]:
path = "./data/"
region = "japan"
filepath = path * region * ".csv"

In [ ]:
japan_full = CSV.read(filepath, DataFrame);
japan = japan_full[japan_full.Datetime .> DateTime(1988,1,1,0,0,0),:];

### Data visualization

### Cube split

In [ ]:
japan, japan_cubes = region_cube_split(japan,side=5,energyRelease=true);

### Seismic Network Creation

In [ ]:
MG = create_network(japan, japan_cubes; edgeWeight=false)
connectivity = degree(MG);

#### Connectivity Distribution

## Motifs

### Motifs Discovery

#### Triangles

In [ ]:
inputName = "./networks/" * region * "_5km.txt"
queryName = "./networks/queryTriangle.txt";

In [ ]:
stats = motifs_discovery(inputName,queryName)

#### Tetrahedrons

In [ ]:
inputName = "./networks/" * region * "_5km.txt"
queryName = "./networks/queryTetrahedron.txt";

In [ ]:
stats = motifs_discovery(inputName,queryName)

### Motifs Analysis

#### Triangles

In [ ]:
motif = "Triangle"
filename= "./motifs" * motif * "_" * region * "_5km.csv"
motifs = readdlm(filename, ',', Int64);
motifs;

In [ ]:
motif_energy = total_mean_energy(motifs, japan, japan_cubes);
areas = area_triangles(motifs, japan_cubes);

In [ ]:
area = [value for value in values(areas)];
totalenergy = [value[1] for value in values(motif_energy)]
meanenergy = [value[2] for value in values(motif_energy)];

#### Area weighted processing

In [ ]:
area_weight_totalenergy = []
area_weight_meanenergy = []
for key in keys(motif_energy)
    # Used to filter out zeros and very small areas (triangles on the vertical for example)
    if areas[key] > 1
        push!(area_weight_totalenergy, areas[key]/motif_energy[key][1])
        push!(area_weight_meanenergy, areas[key]/motif_energy[key][2])
    end
end

#### Tetrahedrons

In [ ]:
filename= "./motifs" * motif * "_" * region * "_5km.csv"
motifs = readdlm(filename, ',', Int64);
motifs;

In [ ]:
motif_energy = total_mean_energy(motifs, japan, japan_cubes)
volumes = volume_tetrahedrons(motifs,japan_cubes);

In [ ]:
volume = [value for value in values(volumes)];
totalenergy = [value[1] for value in values(motif_energy)]
meanenergy = [value[2] for value in values(motif_energy)];

#### Volume weighted processing

In [ ]:
volume_weight_totalenergy = []
volume_weight_meanenergy = []
for key in keys(motif_energy)
    # Used to filter out zeros and very small areas (triangles on the vertical for example)
    if volumes[key] > 1
        push!(volume_weight_totalenergy, volumes[key]/motif_energy[key][1])
        push!(volume_weight_meanenergy, volumes[key]/motif_energy[key][2])
    end
end

### Distributions and Fits